# Instruction-tune Llama 2

Reference: Philipp Schmid https://www.philschmid.de/instruction-tune-llama-2

## Install dependencies

In [1]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.1 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 33.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 120.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 131.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.7/757.7 kB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.9/781.9 kB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 131.1 MB/s eta 0:00:00a 0

## Dataset

Use Databricks Dolly dataset `databricks/databricks-dolly-15k`.

Let's first load the dataset from the hub.

In [2]:
from datasets import load_dataset

# Load the dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1282.66it/s]
                                                        

Dataset json downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


Now take a look at the dataset. The data is in JSON format with the following schema:

```js
{
    'instruction': 'I am trying to book a flight from Singapore to Sydney, what shall I do if the flight is too expensive?', 
    'context': '', 
    'response': 'You will have the option to choose from local Asian low-cost airlines such as Scoot, Jetstar, or AirAsia which would provide cheaper flights options.', 
    'category': 'general_qa'
}
```

In [3]:
from random import randrange

print(f'dataset size: {len(dataset)}')
print(dataset[randrange(len(dataset))])

dataset size: 15011
{'instruction': 'I am trying to book a flight from Singapore to Sydney, what shall I do if the flight is too expensive?', 'context': '', 'response': 'You will have the option to choose from local Asian low-cost airlines such as Scoot, Jetstar, or AirAsia which would provide cheaper flights options.', 'category': 'general_qa'}


Let's define a function to convert the data into a collection of tasks described by instructions.

In [4]:
def format_instructions(sample):
    return f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the Input using an LLM.

### Input:
{sample['response']}

### Response:
{sample['instruction']}
"""

Test the `format_instructions` function with a random sample in the dataset.

In [6]:
from random import randrange

sample_idx = randrange(len(dataset))
print(dataset[sample_idx])
print(format_instructions(dataset[sample_idx]))

{'instruction': 'When would a railway be considered a heritage railway?', 'context': "A heritage railway or heritage railroad (US usage) is a railway operated as living history to re-create or preserve railway scenes of the past. Heritage railways are often old railway lines preserved in a state depicting a period (or periods) in the history of rail transport. The British Office of Rail and Road defines heritage railways as follows:...'lines of local interest', museum railways or tourist railways that have retained or assumed the character and appearance and operating practices of railways of former times. Several lines that operate in isolation provide genuine transport facilities, providing community links. Most lines constitute tourist or educational attractions in their own right. Much of the rolling stock and other equipment used on these systems is original and is of historic value in its own right. Many systems aim to replicate both the look and operating practices of historic f

## Instruction-tune Llama 2

Flash Attention needs NVIDIA Ampere GPUs. See if we have it.

In [8]:
!python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/ec2-user/miniconda3/lib/python3.11/site-packages/torch/cuda/__init__.py", line 381, in get_device_capability
    prop = get_device_properties(device)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ec2-user/miniconda3/lib/python3.11/site-packages/torch/cuda/__init__.py", line 395, in get_device_properties
    _lazy_init()  # will define _get_device_properties
    ^^^^^^^^^^^^
  File "/home/ec2-user/miniconda3/lib/python3.11/site-packages/torch/cuda/__init__.py", line 247, in _lazy_init
    torch._C._cuda_init()
RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx


The driver is missing. See how to install it here: https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/install-nvidia-driver.html

If you need to use a password for `ec2-user`, first change to root with `$ sudo su`, then `sudo passwd ec2-user` and type in a new password twice. After password is created, switch back to `ec2-user` by `$ su ec2-user`.

For getting the driver from AWS (located in S3), add an IAM role to the EC2 which can access S3.

After installation, checking the driver and GPU details should give you this:

```bash
$ nvidia-smi -q | head

==============NVSMI LOG==============

Timestamp                                 : Mon Jul 31 14:13:24 2023
Driver Version                            : 535.54.03
CUDA Version                              : 12.2

Attached GPUs                             : 1
GPU 00000000:00:1E.0
    Product Name                          : NVIDIA A10G
```

Now this check should pass with no complaint and the pip installs would work.

In [1]:
!python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"

In [2]:
!pip install ninja packaging

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 6.6 MB/s eta 0:00:00


In [3]:
!MAX_JOBS=4 pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 54.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [13 lines of output]
      
      
      torch.__version__  = 2.0.1+cu117
      
      
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-6hpibdd5/flash-attn_ec1990251ecc4243a47398f0a6794995/setup.py", line 108, in <module>
          raise_if_cuda_home_none("flash_attn")
        File "/tmp/pip-install-6hpibdd5/flash-attn_ec1990251ecc4243a47398f0a6794995/setup.py", line 55, in raise_if_cuda_home_none
          raise RuntimeError(
      RuntimeError: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com

I need a newer version of CUDA?
 
https://developer.nvidia.com/cuda-downloads?target_os=Linux&target_arch=x86_64&Distribution=CentOS&target_version=7&target_type=runfile_local

Or install cuda with `conda`?

https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html

```bash
$ conda install cuda -c nvidia
```

In [5]:
!MAX_JOBS=4 pip install flash-attn --no-build-isolation

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 56.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 322.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [114 lines of output]
      
      
      torch.__version__  = 2.0.1+cu117
      
      
      fatal: not a git repository (or any parent up to mount point /)
      Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-311
      creating build/lib.linux-x86_64-cpython-311/flash_attn
      copying flash_attn/tmp.py -> build/lib.linux-x86_64-cpython-311/flash_attn
      copying flash_attn/rotary.py -> build/lib.linux-x86_64-cpython-311/

Then I got this.
```bash
The detected CUDA version (12.2) mismatches the version that was used to compile
      PyTorch (11.7). Please make sure to use the same CUDA versions.
```
So let me try install CUDA pytorch 11.7

```bash
# uninstall the current CUDA 12.2 
$ conda remove cuda
# install 11.7
$ conda install cuda -c nvidia/label/cuda-11.7.0
```

After this, I still needed to install `typing-extensions`:

```bash
$ pip install typing-extensions
```

Then it finally started to build 
```bash
Building wheels for collected packages: flash-attn
  Building wheel for flash-attn (setup.py) ... \
```
